# Простая сеть на pytorch для распознования рукописных букв

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision.transforms as tfs
from PIL import Image, ImageFile
from torchvision.datasets import MNIST
import pandas as pd
import numpy as np
from torchvision import models
from torch.nn.functional import cross_entropy
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

In [ ]:
data_tfs = tfs.Compose([tfs.ToTensor(), tfs.Normalize((0.5),(0.5))])

In [ ]:
root = './'
train = MNIST(root, train=True, transform=data_tfs, download=True)
test = MNIST(root, train=False, transform=data_tfs, download=True)

In [ ]:
print(f'Data size:\n\t train {len(train)}, test {len(test)}')
print(f'Data shape:\n\t features {train[0][0].shape},\n\t target {type(test[0][1])}')

In [ ]:
from torch.utils.data import DataLoader

batch_size = 128

train_loader = DataLoader(train, batch_size=batch_size, drop_last=True)
test_loader = DataLoader(test, batch_size=batch_size, drop_last=True)

In [ ]:
x_batch, y_batch = next(iter(train_loader))
x_batch.shape, y_batch.shape

In [ ]:
features = 784
classes = 10

In [ ]:
W = torch.FloatTensor(features, classes).uniform_(-1,1) / features**0.5
W.requires_grad_()

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(torch.cuda.is_available())

In [ ]:
epochs = 3
lr = 1e-2
history = []

In [ ]:
for i in range(epochs):
    for x_batch, y_batch in train_loader:
        x_batch = x_batch.reshape(x_batch.shape[0], -1)
        y_batch = y_batch
        
        logits = x_batch @ W
        probabilities = torch.exp(logits) / torch.exp(logits).sum(dim=1, keepdims=True)
        
        loss = -torch.log(probabilities[range(batch_size), y_batch]).mean()
        history.append(loss.item())
        
        loss.backward()
        
        grad = W.grad
        with torch.no_grad():
            W -= lr * grad
        W.grad.zero_()
        
print(f'{i+1},\t loss: {history[-1]}')

In [ ]:
plt.figure(figsize=(10,7))
plt.plot(history)
plt.title('Loss by batch iterations')
plt.ylabel('Entropy Loss')
plt.xlabel('batches')
plt.show()

In [ ]:
acc = 0
batches = 0

for x_batch, y_batch in test_loader:
    batches += 1
    x_batch = x_batch.view(x_batch.shape[0], -1)
    y_batch = y_batch
    
    preds = torch.argmax(x_batch @ W, dim=1)
    acc += (preds == y_batch).cpu().numpy().mean()
print(f'Test accuracy {acc / batches: .3}')